In [4]:
import requests
import json
import pickle
pokemons = requests.get("https://pokeapi.co/api/v2/pokemon?&limit=1118").json()
with open("./data/pokemons.json", "w+") as f:
  json.dump(pokemons, f)
with open("./data/pokemons.pickle", "wb") as f:
  pickle.dump(pokemons, f)

In [15]:
import requests
import json
import pickle
moves = requests.get("https://pokeapi.co/api/v2/move?limit=844").json()["results"]
with open("./data/moves.json", "w+") as f:
  json.dump(moves, f)
with open("./data/moves.pickle", "wb") as f:
  pickle.dump(moves, f)

In [5]:
import requests
import json
import pickle
moves = requests.get("https://pokeapi.co/api/v2/type").json()["results"]
with open("./data/types.json", "w+") as f:
  json.dump(moves, f)
with open("./data/types.pickle", "wb") as f:
  pickle.dump(moves, f)

In [1]:
import requests
import pickle
ladder = requests.get("https://pokemonshowdown.com/ladder/gen8randombattle.json").json()
ladder["toplist"][0]

{'userid': 'appelsap',
 'username': 'appelsap',
 'w': 2566,
 'l': 1962,
 't': 0,
 'gxe': 90.3,
 'r': 1920.1336212641,
 'rd': 25,
 'sigma': 0,
 'rptime': '1644138000',
 'rpr': 1920.1336212641,
 'rprd': 25.876418629631,
 'rpsigma': 0,
 'elo': 2454.127109616}

In [1]:
def get_user_replays(user: dict) -> list[dict]:
  userid = user["userid"]
  page = 1
  len_replays = 51
  replays_list = []
  while len_replays == 51 and page < 25:
      replays = requests.get(f"https://replay.pokemonshowdown.com/search.json?user={userid}&page={page}&format=gen8randombattle").json()
      replays_list.extend(replays)
      page += 1
      len_replays = len(replays)
  return userid, replays_list

In [3]:
from tqdm import tqdm
from multiprocessing import Pool
from typing import Callable, TypeVar
T = TypeVar("T")
def async_multiprocess_loop(func: Callable[[tuple], T], array: list, callback: Callable[[tuple], None]) -> None:
  pbar = tqdm(total=len(array))
  def update(args):
    callback(args)
    pbar.update()
  pool = Pool()
  for item in array:
      pool.apply_async(func, args=(item,), callback=update)
  pool.close()
  pool.join()
  pbar.close()

In [4]:
user_replays = {}
def update(args):
  userid, replays = args
  user_replays[userid] = replays
async_multiprocess_loop(get_user_replays, ladder["toplist"], update)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
import pickle
with open("./data/user_replays.pickle", "wb") as f:
  pickle.dump(user_replays, f)

In [ ]:
total_replays = 0
for replays in user_replays.values():
  total_replays += len(replays)
total_replays

In [ ]:
replays_ids = set()
for user in user_replays.keys():
    for replay in user_replays[user]:
      try:
        replays_ids.add(replay["id"])
      except:
        print(replay)
len(replays_ids)

In [ ]:
import pickle
with open("./data/replay_ids.pickle", "wb") as f:
  pickle.dump(replays_ids, f)

In [5]:
import pickle
with open("./data/replay_ids.pickle", "rb") as f:
    replay_ids_list = list(pickle.load(f))
def get_replay(replay_id: str) -> dict:
  return requests.get(f"https://replay.pokemonshowdown.com/{replay_id}.json").json()

In [2]:
replays = []
def update(replay):
  replays.append(replay)
# async_multiprocess_loop(get_replay, replay_ids_list, update)
# with open("replays.pickle", "wb") as f:
#  pickle.dump(replays, f)
import pickle
with open("./data/replays.pickle", "rb") as f:
  replays = pickle.load(f)

In [18]:
replays[0]

{'id': 'gen8randombattle-1438712292',
 'p1': 'momma umbridge',
 'p2': 'Mario Pratt',
 'format': 'gen8randombattle',
 'log': "|j|☆momma umbridge\n|j|☆Mario Pratt\n|t:|1634704257\n|gametype|singles\n|player|p1|momma umbridge|170|2062\n|player|p2|Mario Pratt|drayden|2049\n|teamsize|p1|6\n|teamsize|p2|6\n|gen|8\n|tier|[Gen 8] Random Battle\n|rated|\n|rule|Species Clause: Limit one of each Pokémon\n|rule|HP Percentage Mod: HP is shown in percentages\n|rule|Sleep Clause Mod: Limit one foe put to sleep\n|\n|t:|1634704257\n|start\n|switch|p1a: Corsola|Corsola, L92, M|269/269\n|switch|p2a: Starmie|Starmie, L84|238/238\n|turn|1\n|inactive|Battle timer is ON: inactive players will automatically lose when time's up. (requested by Mario Pratt)\n|\n|t:|1634704265\n|switch|p2a: Sigilyph|Sigilyph, L83, F|255/255\n|move|p1a: Corsola|Toxic|p2a: Sigilyph\n|-status|p2a: Sigilyph|tox\n|\n|upkeep\n|turn|2\n|\n|t:|1634704277\n|switch|p1a: Togekiss|Togekiss, L80, M|267/267\n|move|p2a: Sigilyph|Energy Ball|p1a

In [3]:
log: list = replays[0]["log"].strip("|").split("\n")
log = log[log.index("|start") + 1:]
log

['|switch|p1a: Corsola|Corsola, L92, M|269/269',
 '|switch|p2a: Starmie|Starmie, L84|238/238',
 '|turn|1',
 "|inactive|Battle timer is ON: inactive players will automatically lose when time's up. (requested by Mario Pratt)",
 '|',
 '|t:|1634704265',
 '|switch|p2a: Sigilyph|Sigilyph, L83, F|255/255',
 '|move|p1a: Corsola|Toxic|p2a: Sigilyph',
 '|-status|p2a: Sigilyph|tox',
 '|',
 '|upkeep',
 '|turn|2',
 '|',
 '|t:|1634704277',
 '|switch|p1a: Togekiss|Togekiss, L80, M|267/267',
 '|move|p2a: Sigilyph|Energy Ball|p1a: Togekiss',
 '|-resisted|p1a: Togekiss',
 '|-damage|p1a: Togekiss|233/267',
 '|',
 '|upkeep',
 '|turn|3',
 '|',
 '|t:|1634704285',
 '|switch|p2a: Xurkitree|Xurkitree, L76|251/251',
 '|move|p1a: Togekiss|Nasty Plot|p1a: Togekiss',
 '|-boost|p1a: Togekiss|spa|2',
 '|',
 '|upkeep',
 '|turn|4',
 '|',
 '|t:|1634704293',
 '|switch|p1a: Dugtrio|Dugtrio, L82, M|192/192',
 '|move|p2a: Xurkitree|Thunderbolt|p1a: Dugtrio',
 '|-immune|p1a: Dugtrio',
 '|',
 '|upkeep',
 '|turn|5',
 '|',
 '|

In [16]:
p1_team = {}
p2_team = {}
teams = [p1_team, p2_team]
for entry in log:
  items = entry.strip("|").split("|")
  if items[0] == "switch":
    player_index = int(items[1][1:2]) - 1
    pokemon_info = items[2].replace(" ", "").split(",")
    pokemon, level = pokemon_info[0:2]
    level = int(level[1:])
    sex = None
    if len(pokemon_info) == 3:
      sex = pokemon_info[2]
    hp = int(items[3].split("/")[0])
    teams[player_index][pokemon] = {"level": level, "sex": sex, "hp": hp, "moves": set()}
  elif items[0] == "move":
    player_index = int(items[1][1:2]) - 1
    pokemon = items[1][5:]
    move = items[2]
    teams[player_index][pokemon]["moves"].add(move)
teams

[{'Corsola': {'level': 92, 'sex': 'M', 'hp': 269, 'moves': {'Power Gem'}},
  'Togekiss': {'level': 80, 'sex': 'M', 'hp': 233, 'moves': set()},
  'Dugtrio': {'level': 82, 'sex': 'M', 'hp': 192, 'moves': {'Earthquake'}},
  'Garchomp': {'level': 74, 'sex': 'M', 'hp': 282, 'moves': {'Earthquake'}},
  'Scolipede': {'level': 80,
   'sex': 'M',
   'hp': 59,
   'moves': {'Megahorn', 'Spikes'}},
  'Kommo-o': {'level': 80, 'sex': 'F', 'hp': 209, 'moves': {'Max Knuckle'}}},
 {'Starmie': {'level': 84, 'sex': None, 'hp': 238, 'moves': set()},
  'Sigilyph': {'level': 83,
   'sex': 'F',
   'hp': 255,
   'moves': {'Air Slash', 'Energy Ball'}},
  'Xurkitree': {'level': 76,
   'sex': None,
   'hp': 251,
   'moves': {'Max Overgrowth', 'Max Starfall', 'Thunderbolt'}},
  'Yveltal': {'level': 69, 'sex': None, 'hp': 287, 'moves': {'Oblivion Wing'}},
  'Rhyperior': {'level': 82,
   'sex': 'F',
   'hp': 323,
   'moves': {'Earthquake', 'Rock Polish', 'Stone Edge'}}}]

In [9]:
class Observation(object):
  def __init__(self):
    self.p1_team = None
    self.p1_active_pokemon = None
    self.p2_team = None
    self.p2_active_pokemon = None
  
  def to_array(self):
    p1_array = []
    return [self.p1_team, self.p2_team]

In [ ]:
import copy
turn = 0
observations = [[]]
from_player_pov = 0
current_obs = Observation()
for entry in log:
  if items[0] == "switch":
    player_index = int(items[1][1:2]) - 1
    team = copy.deepcopy(teams[player_index])
    if player_index == from_player_pov:
      current_obs.p1_team = team
    else:
      current_obs.p2_team = team
  elif items[0] == "move":
    player_index = int(items[1][1:2]) - 1
    pokemon = items[1][5:]
    move = items[2]
    teams[player_index][pokemon]["moves"].add(move)
  elif items[0] == "turn":
    observations[turn] = current_obs.to_array()
    observations.append([])
    turn += 1

In [ ]:
import requests
from bs4 import BeautifulSoup
r = requests.get("https://replay.pokemonshowdown.com/oumonotype-82345404")
BeautifulSoup(r.text, 'html.parser')

In [4]:
import poke_env

len(poke_env.player.env_player.Gen8EnvSinglePlayer._ACTION_SPACE)

22